In [11]:
import os
import django
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from django.core.exceptions import ObjectDoesNotExist
import time
from bs4 import BeautifulSoup

In [12]:
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'web_restaurant.settings')
django.setup()

In [15]:
from restaurant.models import Restaurant, Review, Chef

# Chrome 드라이버 설정
options = Options()
options.add_argument("--headless")  # 브라우저 창을 띄우지 않고 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 비동기 코드에서 ORM 호출 허용
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# 페이지가 로딩될 시간을 기다림
# time.sleep(3)



# 페이지 스크롤 함수
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # 스크롤 후 대기 시간

        # 새로운 높이 계산 후 비교
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 스크롤이 더 이상 불가능할 때 종료
        last_height = new_height

# <br> 태그 제거 함수
def clean_review_text(text):

    soup = BeautifulSoup(text, 'html.parser')

    # 해시태그 <span class="__hashtag"> 태그들 제거
    for hashtag in soup.find_all('span', class_='__hashtag'):
        hashtag.decompose()


    return re.sub(r'<br\s*/?>', '\n', soup.get_text())  # <br>을 줄바꿈으로 변환



def collect_reviews(restaurant):
    # 평점과 리뷰 내용 수집
    reviews = []
    try:
        # URL 생성 및 페이지 이동
        url = f"https://app.catchtable.co.kr/ct/shop/{restaurant.restaurant_name_en}/review?type=DINING&sortingFilter=L"
        driver.get(url)
        print(f"{restaurant.restaurant_name}의 리뷰 페이지 접근 성공: {url}")

        # 페이지 로딩 대기 및 스크롤
        time.sleep(3)
        scroll_to_bottom()


        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "_10fm75h6"))
        )
        # 모든 평점과 리뷰 요소를 찾음
        ratings = driver.find_elements(By.CLASS_NAME, "_10fm75h6")
        contents = driver.find_elements(By.CLASS_NAME, "review-content")

        for rating, content in zip(ratings, contents):
            # rating 값 정수로 버림
            rating_value = float(rating.text)
            review_type = 'good' if rating_value == 5 else 'bad'
            review_text = clean_review_text(content.get_attribute("innerHTML").strip())

            review_data = {
                "rating": review_type,
                "content": review_text
            }
            reviews.append(review_data)

    except Exception as e:
        print(f"{restaurant.restaurant_name} 리뷰 수집 중 오류 발생: {e}")
    
    return reviews

def save_reviews(restaurant, reviews):
    # 기존 리뷰 삭제 후 새로운 리뷰 저장
    Review.objects.filter(restaurant=restaurant).delete()
    print(f"기존 리뷰 삭제 완료: {restaurant.restaurant_name}")

    for review in reviews:
        Review.objects.create(
            restaurant=restaurant,
            review_text=review['content'],
            review_category=review['rating']
        )
        print(f"저장된 리뷰 - 평점: {review['rating']}, 내용: {review['content']}")



# 메인
restaurants = Restaurant.objects.all()
for restaurant in restaurants:
    print(f"'{restaurant.restaurant_name}' 리뷰 수집 시작")
    reviews = collect_reviews(restaurant)
    if reviews:
        save_reviews(restaurant, reviews)
        print(f"'{restaurant.restaurant_name}' 리뷰 수집 완료")
    else:
        print(f"'{restaurant.restaurant_name}'에 대한 리뷰가 없습니다.")


# 드라이버 종료
driver.quit()
print("모든 리뷰 수집 완료")



'로컬릿x스텔라 고메데이' 리뷰 수집 시작
로컬릿x스텔라 고메데이의 리뷰 페이지 접근 성공: https://app.catchtable.co.kr/ct/shop/localeatstellagourmetday/review?type=DINING&sortingFilter=L
로컬릿x스텔라 고메데이 리뷰 수집 중 오류 발생: Message: 

'로컬릿x스텔라 고메데이'에 대한 리뷰가 없습니다.
'리북방x스텔라 고메데이' 리뷰 수집 시작
리북방x스텔라 고메데이의 리뷰 페이지 접근 성공: https://app.catchtable.co.kr/ct/shop/leebukbangstellagourmetday/review?type=DINING&sortingFilter=L
리북방x스텔라 고메데이 리뷰 수집 중 오류 발생: Message: 

'리북방x스텔라 고메데이'에 대한 리뷰가 없습니다.
'비아톨레도 파스타바' 리뷰 수집 시작
비아톨레도 파스타바의 리뷰 페이지 접근 성공: https://app.catchtable.co.kr/ct/shop/viatoledo/review?type=DINING&sortingFilter=L


/var/folders/_4/mvxsg_zj2q35755ky_zps0780000gn/T/ipykernel_11583/343676565.py:35: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


기존 리뷰 삭제 완료: 비아톨레도 파스타바
저장된 리뷰 - 평점: bad, 내용: 2인방문 3가지 주문 강렬한 짠맛으로 세가지맛이 큰 차이가 없었으며, 불친절 재방문 의사 없음. 예약하기 힘들어서 신청했으나 한타임에 한테이블만 받기 때문에 예약하기 힘들었던걸까? 라는 생각을 해본다 안가본 분들 있다면 가보세요
저장된 리뷰 - 평점: bad, 내용: 일단 음식의 간이 짜도 너무 짭니다…   이탈리아 본토식이라 짠거 아니냐는데 그 수준이 아니라 간을 못 맞추시는거 같아요. 이탈리아 여행도 얼마전 다녀왔고 현지 3스타 식당도 다녀왔어요. 미국에서 몇년 살다와서 짠 음식에도 익숙합니다..  방문 사진에 스타터로 갖가지 핑거푸드가 나오는게 비주얼적으로 눈길을 끌었는데 그마저도 없었고 저 빵 두조각이 나왔네요. 실망스럽
저장된 리뷰 - 평점: bad, 내용: 
저장된 리뷰 - 평점: bad, 내용: 기대보다 좀 아쉬워요. 음식들 전부 너무 간이 쎄서 놀랐어요. 그 외 .. 할많하않..  한번 쯤 가볼만 해요. 
저장된 리뷰 - 평점: bad, 내용: 으음 ~~.ᐟ
저장된 리뷰 - 평점: bad, 내용: 의자 불편.  맛 편차 심함
저장된 리뷰 - 평점: bad, 내용: 가기전에 지인으로부터 간이 좀 짜다고 들어서 사전에 덜 짜게 해달라고 말씀을 드렸는데요. 여전히 까르보나라는 짰습니다. 라구는 짜진 않았지만, 면이 많이 익혀진것 같아 아쉬웠습니다. 그리고 전체적으로 면수가 많이 들어가서 질은 느낌이 있었습니다.  이탈리아 본토의 파스타맛인지는 모르겠지만, 제 입맛에는 맞지않아서 재방문 할 생각은 없습니다..
저장된 리뷰 - 평점: bad, 내용: 좋지는 않았습니다ㅠ  봉골레는 그냥저냥 무난했구요.. 까르보나라는 맛이 너무 엷고 밋밋했습니다...ㅠㅠ  좋아하시는 분들도 매우 많은걸 보면 업장의 방향성이 저와는 안맞는것 같습니다😅
저장된 리뷰 - 평점: bad, 내용: 파스타에 미쳤다!! 하시면 추천드리고 보틀와인 필수입니다 저렴한거먹어도 2인 20뚜딱입니다 파슬리 향거슬리

KeyboardInterrupt: 

In [4]:
from restaurant.models import Review
print(Review)

<class 'restaurant.models.Review'>
